In [4]:
import numpy as np
import pandas as pd

In [5]:
dat = pd.read_csv("final_major_map_waypoints.txt", header=None)

In [6]:
print(dat)

                 0           1            2
0      2599.199951  104.746101  4372.600098
1      2599.199951  104.682404  4372.600098
2      2599.199951  104.594200  4372.600098
3      2599.199951  104.481499  4372.600098
4      2599.199951  104.344292  4372.600098
...            ...         ...          ...
27251  2589.463623  103.452888  4322.631348
27252  2589.454102  103.454468  4322.580078
27253  2589.444824  103.455994  4322.529785
27254  2589.436035  103.457458  4322.481445
27255  2589.427490  103.458893  4322.434570

[27256 rows x 3 columns]


In [7]:
dat_np = np.asarray(dat)

In [8]:
dat_np
print(dat_np.shape)
dat_np = dat_np[:,[0,2,1]]

(27256, 3)


In [28]:
from roar_py_interface import RoarPyWaypoint
from typing import List, Tuple
from roar_py_carla_implementation.utils import *
import carla

def generate_waypoints(locations : List[np.ndarray], distance_between_waypoints : float, lane_width : float):
    locations = [location_from_carla(carla.Location(x=location[0], y=location[1], z=location[2])) for location in locations]
    waypoint_locations = [locations[0]]
    waypoint_rotations = []
    last_location_idx = 0
    for i in range(1,len(locations)):
        next_location = locations[i]
        dist_to_last_location = np.linalg.norm(next_location - locations[last_location_idx])
        if dist_to_last_location > distance_between_waypoints:
            waypoint_locations.append(next_location)
            waypoint_rotations.append(np.arctan2(next_location[1] - locations[last_location_idx][1], next_location[0] - locations[last_location_idx][0]))
            last_location_idx = i
    waypoint_rotations.append(np.arctan2(locations[0][1] - waypoint_locations[-1][1], locations[0][0] - waypoint_locations[-1][0]))
    waypoints = []
    for i in range(len(waypoint_locations)):
        waypoints.append(RoarPyWaypoint(
            waypoint_locations[i],
            np.array([0,0,waypoint_rotations[i]]),
            lane_width
        ))
    return waypoints

waypoints = generate_waypoints(dat_np, 2, 3.5)
print("final length",len(waypoints))

np.savez_compressed("final_major_map_waypoints.npz", **RoarPyWaypoint.save_waypoint_list(waypoints))
print(waypoints[0].location)

final length 6942
[-4372.60009766 -2599.19995117   104.74610138]


In [26]:

def generate_spawn_points(waypoints : List[RoarPyWaypoint]) -> Tuple[np.ndarray, np.ndarray]:
    spawn_point_locations = []
    spawn_rotations = []
    for spawn_point in waypoints[::len(waypoints)//10]:
        spawn_point_locations.append(spawn_point.location)
        spawn_rotations.append(spawn_point.roll_pitch_yaw)
    return np.stack(spawn_point_locations, axis=0), np.stack(spawn_rotations, axis=0)

spawn_point_locations, spawn_rotations = generate_spawn_points(waypoints)
print(spawn_point_locations.shape)
np.savez_compressed("final_major_map_spawn_points.npz", locations=spawn_point_locations, rotations=spawn_rotations)

(11, 3)
